In [13]:
from transformers import BertTokenizer, BertForMaskedLM, AutoTokenizer
import torch
import pandas as pd

In [14]:
df = pd.read_table('/kaggle/input/paranmt/filtered.tsv')
df.head()

,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,0,"If Alkar is flooding her with psychic waste, t...","if Alkar floods her with her mental waste, it ...",0.785171,0.010309,0.014195,0.981983
1,1,Now you're getting nasty.,you're becoming disgusting.,0.749687,0.071429,0.065473,0.999039
2,2,"Well, we could spare your life, for one.","well, we can spare your life.",0.919051,0.268293,0.213313,0.985068
3,3,"Ah! Monkey, you've got to snap out of it.","monkey, you have to wake up.",0.664333,0.309524,0.053362,0.994215
4,4,I've got orders to put her down.,I have orders to kill her.,0.726639,0.181818,0.009402,0.999348


In [15]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
import numpy as np
from transformers import RobertaForSequenceClassification, RobertaTokenizer


tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

def predict_toxicity(texts, device='cpu', clf_name = 's-nlp/roberta_toxicity_classifier_v1'):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    clf = RobertaForSequenceClassification.from_pretrained(clf_name).to(device)
    clf_tokenizer = RobertaTokenizer.from_pretrained(clf_name)
    with torch.inference_mode():
        inputs = clf_tokenizer(texts, return_tensors='pt', padding=True).to(clf.device)
        out = torch.softmax(clf(**inputs).logits, -1)[:, 1].cpu().numpy()
    return out


def mask_toxic(sentence, threshold=0.3):
    words = sentence.split()
    probabilities = predict_toxicity(words)
    text_prep = []
    toxic_indexes = []
    for _word, _prob in zip(words, probabilities):
        if _prob > threshold:
            text_prep.append("[MASK]")
        else:
            text_prep.append(_word)
    text_prep = " ".join(text_prep)
    tokenized = tokenizer(text_prep, return_tensors="pt")
    return tokenized

def get_mask_index(inputs):
    mask_token_indexes = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    return mask_token_indexes

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
def infer(sentence):
    inputs = mask_toxic(sentence)
    with torch.no_grad():
        logits = model(**inputs).logits
    mask_indexes = get_mask_index(inputs)
    for mask_token_index in mask_indexes:
        predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
        inputs.input_ids[0][mask_token_index] = predicted_token_id
    return inputs

In [18]:
inputs = infer("I hate idiots and you")
tokenizer.decode(inputs.input_ids[0][1:-1])

Some weights of the model checkpoint at s-nlp/roberta_toxicity_classifier_v1 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


'i hate you and you'

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['You are not good']